In [1]:
import os 
import pandas as pd
import numpy as np
from collections import defaultdict

lowes_dir = "/home/jupyter/unity_jointly_rec_and_search/datasets/unified_user/"
lowes_search_path = os.path.join(lowes_dir, "train_searchs.csv")
lowes_sim_path = os.path.join(lowes_dir, "train_sim_recs.csv")
lowes_compl_path = os.path.join(lowes_dir, "train_compl_recs.csv")

with open(lowes_search_path) as fin:
    lowes_search = pd.read_csv(fin, index_col=0)
with open(lowes_sim_path) as fin:
    lowes_sim = pd.read_csv(fin, index_col=0)
with open(lowes_compl_path) as fin:
    lowes_compl = pd.read_csv(fin, index_col=0)
    

/opt/conda/lib/python3.7/site-packages/numpy/lib/arraysetops.py:580: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


In [2]:
lowes_search["rel_pids"] = lowes_search["rel_pids"].apply(lambda x: eval(x))
lowes_sim["sim_pids"] = lowes_sim["sim_pids"].apply(lambda x: eval(x))
lowes_compl["compl_pids"] = lowes_compl["compl_pids"].apply(lambda x: eval(x))

qid_to_relpids = defaultdict(set)
for qid, relpids in zip(lowes_search.qid, lowes_search.rel_pids):
    qid_to_relpids[qid].update(relpids)
aid_to_simpids = defaultdict(set)
for aid, simpids in zip(lowes_sim.aid, lowes_sim.sim_pids):
    aid_to_simpids[aid].update(simpids)
aid_to_complpids = defaultdict(set)
for aid, complpids in zip(lowes_compl.aid, lowes_compl.compl_pids):
    aid_to_complpids[aid].update(complpids)

print("unique qids, pids for search, average length = {:,}, {:,}, {:.3f}".format(len(lowes_search.qid.unique()), 
                                                            len(lowes_search.explode("rel_pids").rel_pids.unique()),
                                                                              np.mean([len(xs) for xs in qid_to_relpids.values()])))
print("unique aids, pids for sim_rec, average length = {:,}, {:,}, {:.3f}".format(len(lowes_sim.aid.unique()), 
                                                          len(lowes_sim.explode("sim_pids").sim_pids.unique()),
                                                         np.mean([len(xs) for xs in aid_to_simpids.values()])))
print("unique aids, pids for compl_rec, average length = {:,}, {:,}, {:.3f}".format(len(lowes_compl.aid.unique()), 
                                                            len(lowes_compl.explode("compl_pids").compl_pids.unique()),
                                                           np.mean([len(xs) for xs in aid_to_complpids.values()])))
lowes_relpids = set([x for xs in qid_to_relpids.values() for x in xs])
lowes_simpids = set([x for xs in aid_to_simpids.values() for x in xs])
lowes_complpids = set([x for xs in aid_to_complpids.values() for x in xs])
lowes_simpids.update(aid_to_simpids.keys())
lowes_complpids.update(aid_to_complpids.keys())
print("relpids and simpids intersect rate = {:.3f}".format(
    len(lowes_relpids.intersection(lowes_simpids)) / len(lowes_relpids.union(lowes_simpids))
))
print("relpids and complpids intersect rate = {:.3f}".format(
    len(lowes_relpids.intersection(lowes_complpids)) / len(lowes_relpids.union(lowes_complpids))
))
print("simpids and complpids intersect rate = {:.3f}".format(
   len( lowes_simpids.intersection(lowes_complpids)) / len(lowes_simpids.union(lowes_complpids))
))

unique qids, pids for search, average length = 633,449, 277,061, 3.819
unique aids, pids for sim_rec, average length = 97,888, 71,278, 3.417
unique aids, pids for compl_rec, average length = 18,543, 15,606, 3.108
relpids and simpids intersect rate = 0.337
relpids and complpids intersect rate = 0.084
simpids and complpids intersect rate = 0.168


In [4]:
import pickle as pkl
esci_dir = "/home/jupyter/unity_jointly_rec_and_search/datasets/amazon_esci_dataset/task_2_multiclass_product_classification/"

with open(os.path.join(esci_dir, "train_qid_to_relpids.pkl"), "rb") as fin:
    esci_qid_to_relpids = pkl.load(fin)
with open(os.path.join(esci_dir, "train_aid_to_simpids.pkl"), "rb") as fin:
    esci_aid_to_simpids = pkl.load(fin)
with open(os.path.join(esci_dir, "train_aid_to_complpids.pkl"), "rb") as fin:
    esci_aid_to_complpids = pkl.load(fin)
esci_relpids = set([x for xs in esci_qid_to_relpids.values() for x in xs])
esci_simpids = set([x for xs in esci_aid_to_simpids.values() for x in xs])
esci_complpids = set([x for xs in esci_aid_to_complpids.values() for x in xs])
    
print("unique qids, pids for search, average length = {:,}, {:,}, {:.3f}".format(len(esci_qid_to_relpids.keys()), 
                                                                            len(esci_relpids),
                                                                              np.mean([len(xs) for xs in esci_qid_to_relpids.values()])))
print("unique aids, pids for sim_rec, average length = {:,}, {:,}, {:.3f}".format(len(esci_aid_to_simpids.keys()), 
                                                                            len(esci_simpids),
                                                                             np.mean([len(xs) for xs in esci_aid_to_simpids.values()])))
print("unique aids, pids for compl_rec, average length = {:,}, {:,}, {:.3f}".format(len(esci_aid_to_complpids.keys()), 
                                                                            len(esci_complpids),
                                                                           np.mean([len(xs) for xs in esci_aid_to_complpids.values()])))

esci_simpids.update(esci_aid_to_simpids.keys())
esci_complpids.update(esci_aid_to_complpids.keys())

print("relpids and simpids intersect rate = {:.3f}".format(
    len(esci_relpids.intersection(esci_simpids)) / len(esci_relpids.union(esci_simpids))
))
print("relpids and complpids intersect rate = {:.3f}".format(
    len(esci_relpids.intersection(esci_complpids)) / len(esci_relpids.union(esci_complpids))
))
print("simpids and complpids intersect rate = {:.3f}".format(
   len(esci_simpids.intersection(esci_complpids)) / len(esci_simpids.union(esci_complpids))
))

unique qids, pids for search, average length = 54,511, 557,840, 12.822
unique aids, pids for sim_rec, average length = 315,575, 216,082, 5.718
unique aids, pids for compl_rec, average length = 82,580, 26,274, 2.939
relpids and simpids intersect rate = 0.379
relpids and complpids intersect rate = 0.127
simpids and complpids intersect rate = 0.107


In [9]:
path = os.path.join(esci_dir, "unified_train")
train_query =  os.path.join(esci_dir, "unified_train/max10_qorient_q2p.scbnegs.train.tsv")
train_sim = os.path.join(esci_dir, "unified_train/a2sp.train.tsv")
train_compl = os.path.join(esci_dir, "unified_train/a2cp.train.tsv")
! wc -l $train_query
! wc -l $train_sim 
! wc -l $train_compl

452082 /home/jupyter/unity_jointly_rec_and_search/datasets/amazon_esci_dataset/task_2_multiclass_product_classification/unified_train/max10_qorient_q2p.scbnegs.train.tsv
1067742 /home/jupyter/unity_jointly_rec_and_search/datasets/amazon_esci_dataset/task_2_multiclass_product_classification/unified_train/a2sp.train.tsv
184362 /home/jupyter/unity_jointly_rec_and_search/datasets/amazon_esci_dataset/task_2_multiclass_product_classification/unified_train/a2cp.train.tsv
